In [1]:
from keras.layers import InputLayer
from keras.models import Sequential
from keras.layers import TimeDistributed
from keras.layers.convolutional import Conv2D
from keras.layers import Dense
from keras.layers.convolutional import MaxPooling2D
from keras.layers import Flatten
from keras.layers import LSTM
from keras.layers import Dense
from keras.models import load_model
from keras.models import model_from_json

import time
import datetime
from datetime import datetime
from keras.layers import Activation, Dense,Dropout
from keras.layers import BatchNormalization
from keras.layers import GlobalAveragePooling2D
import keras
from tensorflow.keras.optimizers import SGD



In [2]:
# Load the TensorBoard notebook extension
%load_ext tensorboard

In [3]:
batch_size = 32
frame_sequence = 8

In [4]:
model = Sequential()
#input layer
model.add(
    TimeDistributed(
        Conv2D(64,(3,3),padding='same', strides=(2,2), activation='relu'),
      input_shape=(frame_sequence, 224, 224, 3)
    )
)

# first conv, 64
model.add(
    TimeDistributed( 
        Conv2D(64, (3,3), 
            padding='same', strides=(2,2), activation='relu')
    )
)

#pooling
model.add(
    TimeDistributed(
        MaxPooling2D((2,2), strides=(2,2))
    )
)

# Second conv, 128
model.add(
    TimeDistributed(
        Conv2D(128, (3,3),
            padding='same', strides=(2,2), activation='relu')
    )
)

 
#pooling
model.add(
    TimeDistributed(
        MaxPooling2D((2,2), strides=(2,2))
    )
)

model.add(TimeDistributed(Flatten()))


model.add(
    LSTM(512, activation='relu', return_sequences=True)
)
model.add(Dropout(0.5))
model.add(Dense(256, activation='relu'))
model.add(Dropout(.3))
model.add(Dense(5, activation='softmax'))

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 time_distributed (TimeDistr  (None, 8, 112, 112, 64)  1792      
 ibuted)                                                         
                                                                 
 time_distributed_1 (TimeDis  (None, 8, 56, 56, 64)    36928     
 tributed)                                                       
                                                                 
 time_distributed_2 (TimeDis  (None, 8, 28, 28, 64)    0         
 tributed)                                                       
                                                                 
 time_distributed_3 (TimeDis  (None, 8, 14, 14, 128)   73856     
 tributed)                                                       
                                                                 
 time_distributed_4 (TimeDis  (None, 8, 7, 7, 128)     0

In [5]:
from TimeDistributedImageDataGenerator import TimeDistributedImageDataGenerator
datagen = TimeDistributedImageDataGenerator.TimeDistributedImageDataGenerator(time_steps = 8)

In [6]:
# prepare an iterators for each dataset
train_it = datagen.flow_from_directory('Dataset/train_set/', class_mode='categorical' ,color_mode='rgb', target_size=(224, 224) , batch_size=batch_size)#set as training data
val_it = datagen.flow_from_directory('Dataset/val_set/', class_mode='categorical' ,color_mode='rgb' , target_size=(224, 224) , batch_size=batch_size )#set as validation data
#test_it = datagen.flow_from_directory('test_set/', class_mode='categorical' ,color_mode='rgb', target_size=(64, 64) , batch_size=batch_size, shuffle=False,     seed=42)#set as training data

Found 6816 images belonging to 5 classes.
Found 2267 images belonging to 5 classes.


In [7]:
#to find the batch and dim of the loading images
batchX, batchy = train_it.next()
print('Batch shape=%s, min=%.3f, max=%.3f' % (batchX.shape, batchX.min(), batchX.max()))

Batch shape=(32, 8, 224, 224, 3), min=0.000, max=255.000


In [8]:
train_it.class_indices.keys()

dict_keys(['blast', 'blood', 'fight', 'gunshots', 'normal'])

In [9]:
import tensorflow
opt = tensorflow.keras.optimizers.SGD(lr=0.0001)
model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=["accuracy"])
#model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

C:\Anaconda3\envs\gpu1\lib\site-packages\keras\optimizer_v2\gradient_descent.py:102: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


In [10]:
from keras.callbacks import ModelCheckpoint
# define checkpoint callback
filepath = 'saved_models/model-ep{epoch:02d}-loss{loss:.3f}-val_loss{val_loss:.3f}.h5'
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, mode='min')

In [11]:
#enter the weights of the model from where you start resuming incase the training is not going to be completed in one pass
model.load_weights('saved_models/model-ep50-loss0.306-val_loss0.235.h5')

In [12]:
logdir="logs/fit/" + datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = keras.callbacks.TensorBoard(log_dir=logdir)

start_time = time.perf_counter()

history=model.fit_generator(
        train_it,
        steps_per_epoch=None,# nb_training_samples // batch_size,
        initial_epoch=50,     #from where you start resuming epochs
        epochs=100,        # the number of epochs to which it runs
        validation_data=val_it,
        validation_steps=None,         # nb_validation_samples // batch_size,
        callbacks=[tensorboard_callback,checkpoint],
        verbose = 1)

print(time.perf_counter()-start_time)


# save model and architecture to single file
# serialize model to JSON
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("best_model1.h5")
print("Saved model to disk")

C:\Anaconda3\envs\gpu1\lib\site-packages\ipykernel_launcher.py:14: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  


Epoch 51/100
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
213/213 [==============================] - ETA: 0s - loss: 0.3010 - accuracy: 0.8901WARNING:tensorflow:AutoGraph could not transform <function Model.make_test_function.<locals>.test_function at 0x00000269D9462F28> and will run it as-is.
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=1

Epoch 72/100
213/213 [==============================] - ETA: 0s - loss: 0.2102 - accuracy: 0.9248
Epoch 72: saving model to saved_models\model-ep72-loss0.210-val_loss0.186.h5
213/213 [==============================] - 459s 2s/step - loss: 0.2102 - accuracy: 0.9248 - val_loss: 0.1864 - val_accuracy: 0.9388
Epoch 73/100
213/213 [==============================] - ETA: 0s - loss: 0.2082 - accuracy: 0.9251
Epoch 73: saving model to saved_models\model-ep73-loss0.208-val_loss0.184.h5
213/213 [==============================] - 460s 2s/step - loss: 0.2082 - accuracy: 0.9251 - val_loss: 0.1836 - val_accuracy: 0.9416
Epoch 74/100
213/213 [==============================] - ETA: 0s - loss: 0.2041 - accuracy: 0.9274
Epoch 74: saving model to saved_models\model-ep74-loss0.204-val_loss0.184.h5
213/213 [==============================] - 461s 2s/step - loss: 0.2041 - accuracy: 0.9274 - val_loss: 0.1841 - val_accuracy: 0.9412
Epoch 75/100
213/213 [==============================] - ETA: 0s - loss: 0.2016 

Epoch 99/100
213/213 [==============================] - ETA: 0s - loss: 0.1484 - accuracy: 0.9482
Epoch 99: saving model to saved_models\model-ep99-loss0.148-val_loss0.161.h5
213/213 [==============================] - 450s 2s/step - loss: 0.1484 - accuracy: 0.9482 - val_loss: 0.1606 - val_accuracy: 0.9495
Epoch 100/100
213/213 [==============================] - ETA: 0s - loss: 0.1466 - accuracy: 0.9480
Epoch 100: saving model to saved_models\model-ep100-loss0.147-val_loss0.159.h5
213/213 [==============================] - 451s 2s/step - loss: 0.1466 - accuracy: 0.9480 - val_loss: 0.1585 - val_accuracy: 0.9509
25662.487069100003
Saved model to disk


In [13]:
# load json and create model
json_file = open('model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("best_model1.h5")
print("Loaded model from disk")

Loaded model from disk


In [ ]:
import tensorflow
opt = tensorflow.keras.optimizers.SGD(lr=0.001)

loaded_model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=["accuracy"])

In [ ]:
import matplotlib.pyplot as plt

# list all data in history
print(history.history.keys())

# summarize history for accuracy
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val_test'], loc='upper left')
plt.show()


# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'Val_test'], loc='upper left')
plt.show()


In [ ]:
#evaluate the model
model.evaluate_generator(generator=val_it,
steps=300)

In [ ]:
#predict the output 

test_it.reset()
pred=model.predict_generator(test_it,
steps=324,
verbose=1)

In [ ]:
predicted_class_indices=np.argmax(pred,axis=1)